In [ ]:
# Download the dataset from Google drive
url_path = 'https://drive.google.com/drive/u/0/folders/1R17xoqkLWiFjHCCftBO_Gvtofu_5Hsjc'
!gdown --id 1R17xoqkLWiFjHCCftBO_Gvtofu_5Hsjc --folder -O data

In [ ]:
# Download the model
!curl -L -o model.tar.gz  https://www.kaggle.com/api/v1/models/google/speciesnet/keras/v4.0.0a/3/download

!tar -xzf model.tar.gz -C model

In [ ]:
# Run the model
!python -m speciesnet.scripts.run_model --folders "data"   --predictions_json "impala_malawi.json"   --model "/model"


In [ ]:
import json
with open('impala_malawi.json') as f:
  output_file = json.load(f)

In [ ]:
import pandas as pd
output_df = pd.json_normalize(output_file['predictions'])

In [ ]:
data = [output_df['prediction'].iloc[i].split(';')[1:] for i in range(len(output_df))]

columns = ['Class', 'Order', 'Family', 'Genus', 'Species', 'Common Name']
df = pd.DataFrame(data, columns=columns)

# replace the '' with Unknown
df = df.replace('', 'Unknown')

df_counts = df.groupby(columns).size().reset_index(name='Count')
df_counts.fillna('Unknown', inplace=True)

In [ ]:
import plotly.express as px

fig = px.treemap(
    df_counts,
    path=columns,
    values='Count',
    title='Hierarchical Animal Classification Treemap'
)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()
